# Notice

To run this notebook, you need to install pipenv via

```
pipenv install --dev
```

In [1]:
import sys
import os
current_dir = os.getcwd()
REPO_BASE_DIR = os.path.dirname(os.path.abspath(current_dir))
sys.path.append(REPO_BASE_DIR)

from dotenv import load_dotenv
load_dotenv(os.path.join(REPO_BASE_DIR, ".env"))

True

In [2]:
from biodsa.agents import DSWizardAgent
agent = DSWizardAgent(
    model_name="gpt-5",
    small_model_name="gpt-5-mini",
    api_type="azure",
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
)

2025-12-23 14:24:01,123 - INFO - Installing biodsa.tools module in sandbox...
2025-12-23 14:24:01,394 - INFO - Uploaded biodsa.tools module to sandbox
2025-12-23 14:24:01,460 - INFO - Successfully extracted biodsa.tools module
2025-12-23 14:24:01,556 - INFO - Created .pth file at /usr/local/lib/python3.12/site-packages/biodsa_tools.pth
2025-12-23 14:24:01,557 - INFO - biodsa.tools module installed in sandbox at /workdir/biodsa
2025-12-23 14:24:01,557 - INFO - You can now use 'from biodsa.tools import xxx' in your sandbox code
2025-12-23 14:24:01,557 - INFO - Sandbox initialized successfully and biodsa.tools installed


In [3]:
agent.register_workspace(
    os.path.join(REPO_BASE_DIR, "biomedical_data/cBioPortal/datasets/acbc_mskcc_2015")
)
execution_results = agent.go("Make a clustering of the patients based on their genomic mutation data to maximize the separation of the prognostic survival outcomes.")
print(execution_results)


2025-12-23 14:24:01,819 - INFO - Installing biodsa.tools module in sandbox...
2025-12-23 14:24:02,053 - INFO - Uploaded biodsa.tools module to sandbox
2025-12-23 14:24:02,116 - INFO - Successfully extracted biodsa.tools module
2025-12-23 14:24:02,209 - INFO - Created .pth file at /usr/local/lib/python3.12/site-packages/biodsa_tools.pth
2025-12-23 14:24:02,209 - INFO - biodsa.tools module installed in sandbox at /workdir/biodsa
2025-12-23 14:24:02,209 - INFO - You can now use 'from biodsa.tools import xxx' in your sandbox code
2025-12-23 14:24:02,232 - INFO - Uploaded table: /Users/zifeng/Documents/github/BioDSA-dev/biomedical_data/cBioPortal/datasets/acbc_mskcc_2015/data_gene_panel_matrix.csv

Uploaded table: /Users/zifeng/Documents/github/BioDSA-dev/biomedical_data/cBioPortal/datasets/acbc_mskcc_2015/data_mutations.csv

Uploaded table: /Users/zifeng/Documents/github/BioDSA-dev/biomedical_data/cBioPortal/datasets/acbc_mskcc_2015/data_clinical_patient.csv

Uploaded table: /Users/zifeng/

----------------------------------------------------------------------------------------------------
human: 

Make a clustering of the patients based on their genomic mutation data to maximize the separation of the prognostic survival outcomes.




2025-12-23 14:24:08,058 - INFO - HTTP Request: POST https://zifeng-gpt-2.openai.azure.com/openai/deployments/gpt-5/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Executing tool: code_execution with input: {'code': "import pandas as pd\nimport numpy as np\nprint('Files to load:')\npaths=['/workdir/data_gene_panel_matrix.csv','/workdir/data_mutations.csv','/workdir/data_clinical_patient.csv','/workdir/data_cna.csv','/workdir/data_sv.csv','/workdir/data_clinical_sample.csv']\nprint(paths)\ndfs={}\nfor p in paths:\n    try:\n        df=pd.read_csv(p)\n        dfs[p]=df\n        print(p, df.shape)\n        print(df.dtypes.head())\n        print(df.head(3))\n    except Exception as e:\n        print('Error loading',p,e)\nprint('Columns overview:')\nfor p,df in dfs.items():\n    print(p, list(df.columns))"}


2025-12-23 14:24:17,543 - INFO - HTTP Request: POST https://zifeng-gpt-2.openai.azure.com/openai/deployments/gpt-5/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


...

----------------------------------------------------------------------------------------------------
ai: 

The analysis plan is complete.




2025-12-23 14:25:07,864 - INFO - HTTP Request: POST https://zifeng-gpt-2.openai.azure.com/openai/deployments/gpt-5/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Executing tool: code_execution with input: {'code': "import pandas as pd\nimport numpy as np\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.decomposition import TruncatedSVD, PCA\nfrom sklearn.cluster import KMeans, AgglomerativeClustering\nfrom sklearn.metrics import silhouette_score, davies_bouldin_score, adjusted_rand_score\nfrom sklearn.feature_selection import VarianceThreshold\nfrom lifelines import KaplanMeierFitter, CoxPHFitter\nfrom lifelines.statistics import logrank_test\nfrom lifelines.utils import concordance_index\nfrom scipy.stats import rankdata\nimport itertools\nimport warnings\nprint('Library versions:')\nimport sklearn, lifelines, scipy\nprint('pandas', pd.__version__)\nprint('numpy', np.__version__)\nprint('sklearn', sklearn.__version__)\nprint('lifelines', lifelines.__version__)\nprint('scipy', scipy.__version__)\nprint('Loading data')\nmut_path='/workdir/data_mutations.csv'\npat_path='/workdir/data_clinical_patient.csv'\nsamp_path='/workdir/data_

2025-12-23 14:25:23,069 - INFO - HTTP Request: POST https://zifeng-gpt-2.openai.azure.com/openai/deployments/gpt-5/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


----------------------------------------------------------------------------------------------------
ai: 

I executed the full analysis plan to cluster patients by genomic mutations and evaluate separation of survival outcomes.

Key steps and results:
- Built a patient-by-gene binary mutation matrix from data_mutations.csv, mapping Tumor_Sample_Barcode to PATIENT_ID via data_clinical_sample.csv, filtering to impactful nonsynonymous Variant_Classification.
- Added TMB_NONSYNONYMOUS per patient.
- Selected genes by prevalence given small sample size (12 patients), resulting in 3 genes plus TMB as features.
- Scaled features and applied dimensionality reduction (TruncatedSVD) to 3 components.
- Evaluated k from 2 to 5 using silhouette and Davies–Bouldin metrics for KMeans and Agglomerative; best performance corresponded to k=4 for both, and the chosen algorithm was Agglomerative (equal metrics, tie broken by implementation).
- Cluster sizes: [2, 2, 2, 6].
- Survival preparation used FOLLO

In [4]:
# Download artifacts separately
# Note: only when you have docker installed and running you can do this, otherwise it will raise an error
print(execution_results.download_artifacts(output_dir="test_artifacts"))


2025-12-23 14:25:23,278 - INFO - Downloaded biodsa to test_artifacts
2025-12-23 14:25:23,286 - INFO - Downloaded biodsa_tools.tar.gz to test_artifacts
2025-12-23 14:25:23,289 - INFO - Downloaded data_clinical_patient.csv to test_artifacts
2025-12-23 14:25:23,293 - INFO - Downloaded data_clinical_sample.csv to test_artifacts
2025-12-23 14:25:23,297 - INFO - Downloaded data_cna.csv to test_artifacts
2025-12-23 14:25:23,301 - INFO - Downloaded data_gene_panel_matrix.csv to test_artifacts
2025-12-23 14:25:23,305 - INFO - Downloaded data_mutations.csv to test_artifacts
2025-12-23 14:25:23,308 - INFO - Downloaded data_sv.csv to test_artifacts
2025-12-23 14:25:23,312 - INFO - Downloaded patient_cluster_assignments.csv to test_artifacts
2025-12-23 14:25:23,316 - INFO - Downloaded selected_genes.csv to test_artifacts
2025-12-23 14:25:23,320 - INFO - Downloaded survival_summary.csv to test_artifacts


['biodsa', 'biodsa_tools.tar.gz', 'data_clinical_patient.csv', 'data_clinical_sample.csv', 'data_cna.csv', 'data_gene_panel_matrix.csv', 'data_mutations.csv', 'data_sv.csv', 'patient_cluster_assignments.csv', 'selected_genes.csv', 'survival_summary.csv']


In [5]:
# Generate PDF report following the structured format:
# 1. User query
# 2. Agent exploration trajectories (messages only, no code)
# 3. Final response with embedded artifacts
# 4. Supplementary materials with code blocks and execution results
print(execution_results.to_pdf(output_dir="test_artifacts"))


2025-12-23 14:25:23,524 - INFO - Downloaded biodsa to /var/folders/rb/nj5lt0x53pj4nt6j_b459_p80000gn/T/biodsa_artifacts_maad56nb
2025-12-23 14:25:23,532 - INFO - Downloaded biodsa_tools.tar.gz to /var/folders/rb/nj5lt0x53pj4nt6j_b459_p80000gn/T/biodsa_artifacts_maad56nb
2025-12-23 14:25:23,535 - INFO - Downloaded data_clinical_patient.csv to /var/folders/rb/nj5lt0x53pj4nt6j_b459_p80000gn/T/biodsa_artifacts_maad56nb
2025-12-23 14:25:23,538 - INFO - Downloaded data_clinical_sample.csv to /var/folders/rb/nj5lt0x53pj4nt6j_b459_p80000gn/T/biodsa_artifacts_maad56nb
2025-12-23 14:25:23,542 - INFO - Downloaded data_cna.csv to /var/folders/rb/nj5lt0x53pj4nt6j_b459_p80000gn/T/biodsa_artifacts_maad56nb
2025-12-23 14:25:23,546 - INFO - Downloaded data_gene_panel_matrix.csv to /var/folders/rb/nj5lt0x53pj4nt6j_b459_p80000gn/T/biodsa_artifacts_maad56nb
2025-12-23 14:25:23,549 - INFO - Downloaded data_mutations.csv to /var/folders/rb/nj5lt0x53pj4nt6j_b459_p80000gn/T/biodsa_artifacts_maad56nb
2025-12-2

test_artifacts/execution_report_20251223_142523.pdf


In [6]:
# Cleanup
agent.clear_workspace()

2025-12-23 14:25:23,785 - INFO - Removed 6 artifacts of 6
2025-12-23 14:25:23,928 - INFO - Successfully pruned unused volumes
2025-12-23 14:25:23,929 - INFO - Sandbox stopped and resources cleaned up


6b630866fb75
6b630866fb75


True